In [ ]:
%%file latch.py

import numpy as np
import matplotlib.pyplot as plt

import pyNN.hardware.spikey as pynn

pynn.setup(mappingOffset=0, calibSynDrivers=False)

spike_train_exc = np.array([100.0, 700.0])
spike_train_inh = np.array([500.0, 800.0])

stim_exc = pynn.Population( 5, pynn.SpikeSourceArray, {'spike_times': spike_train_exc})
stim_inh = pynn.Population(10, pynn.SpikeSourceArray, {'spike_times': spike_train_inh})

neurons = pynn.Population(5, pynn.IF_facets_hardware1)

neurons.record()

proj_exc = pynn.Projection(
    stim_exc,
    neurons,
    pynn.AllToAllConnector(weights=15*pynn.minExcWeight()),
    target="excitatory"
)

proj_inh = pynn.Projection(
    stim_inh,
    neurons,
    pynn.AllToAllConnector(weights=15*pynn.minInhWeight()),
    target="inhibitory"
)

proj_recurrent = pynn.Projection(
    neurons,
    neurons,
    pynn.FixedProbabilityConnector(0.8, weights=9*pynn.minExcWeight()),
    target="excitatory"
)

pynn.run(1000.0)

spikes = neurons.getSpikes()

pynn.end()

fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

for nrn in np.unique(spikes[:,0]):
    mask = (spikes[:,0] == nrn)
    spike_times = spikes[mask,1]
    ax.plot(spike_times, np.ones_like(spike_times) * nrn, "|", ms=5)

ax.set_xlabel(r"time [\si{\milli\second}]")
ax.set_ylabel(r"neuron index")

ax.set_yticks(np.arange(0, 5, 1))

fig.savefig("latch.png")

In [ ]:
from IPython.display import Image
! srun --gres=station508 python latch.py
Image(filename="latch.png")